## Lab 2: 메모리를 추가하여 에이전트 개인화하기

### 개요

Lab 1에서는 로컬 세션에서 단일 사용자에 대해 잘 작동하는 고객 지원 에이전트를 구축했습니다. 하지만 실제 고객 지원은 로컬 환경에서 실행되는 단일 사용자를 넘어 확장되어야 합니다.

**프로덕션 환경에서 에이전트**를 실행할 때는 다음이 필요합니다:
- **다중 사용자 지원**: 수천 명의 고객을 동시에 처리
- **지속적인 저장소**: 세션 수명 주기를 넘어 대화 저장
- **장기 학습**: 고객의 선호도 및 행동 패턴 추출
- **교차 세션 연속성**: 다른 상호 작용 전반에 걸쳐 고객 기억

**워크샵 진행 상황:**
- **Lab 1 (완료)**: 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **Lab 2 (현재)**: 메모리 기능 강화 - 대화 맥락 및 개인화 추가
- **Lab 3**: 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구를 안전하게 공유
- **Lab 4**: 프로덕션 배포 - 관측 기능을 갖춘 AgentCore 런타임 사용
- **Lab 5**: 에이전트 성능 평가 - 온라인 평가로 품질 모니터링
- **Lab 6**: 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성

이 랩에서는 금붕어 기억력(몇 초 만에 대화를 잊어버림)을 가진 에이전트를 똑똑한 개인화된 어시스턴트로 변모시키는 누락된 지속성 및 학습 계층을 추가합니다.

메모리는 지능의 핵심 구성 요소입니다. 대규모 언어 모델(LLM)은 인상적인 기능을 갖추고 있지만 대화 전반에 걸친 지속적인 메모리가 부족합니다. [Amazon Bedrock AgentCore Memory](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory-getting-started.html)는 AI 에이전트가 시간이 지나도 맥락을 유지하고, 중요한 사실을 기억하며, 일관되고 개인화된 경험을 제공할 수 있도록 하는 관리형 서비스를 제공하여 이러한 한계를 해결합니다.

AgentCore Memory는 두 가지 수준에서 작동합니다:
- **단기 메모리 (Short-Term Memory)**: 단일 상호 작용 또는 밀접하게 관련된 세션 내에서 연속성을 제공하는 즉각적인 대화 맥락 및 세션 기반 정보입니다.
- **장기 메모리 (Long-Term Memory)**: 시간이 지남에 따라 개인화된 경험을 가능하게 하는 사실, 선호도 및 요약을 포함하여 여러 대화에서 추출되고 저장된 지속적인 정보입니다.

### Lab 2 아키텍처
<div style="text-align:left">
    <img src="images/architecture_lab2_memory.png" width="75%"/>
</div>

*지속적인 단기 및 장기 메모리 기능을 갖춘 다중 사용자 에이전트.*

### 사전 필수 조건

* 적절한 권한이 있는 **AWS 계정**
* 로컬에 설치된 **Python 3.10+**
* 자격 증명으로 구성된 **AWS CLI**
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)에서 활성화된 **Anthropic Claude 3.7**
* 다음 셀에서 설치될 **Strands Agents** 및 기타 라이브러리
* AWS 워크샵 계정 내에 생성된 리소스:
    - AWS Lambda 함수 
    - AWS Lambda 실행 IAM 역할
    - AgentCore Gateway IAM 역할
    - AWS Lambda 함수에서 사용하는 DynamoDB 테이블 
    - Cognito 사용자 풀 및 사용자 풀 클라이언트


### 단계 1: 라이브러리 가져오기

AgentCore Memory용 라이브러리를 가져와 봅시다. 이를 위해 AgentCore 기능을 사용하는 데 도움이 되는 경량 래퍼인 [Amazon Bedrock AgentCore Python SDK](https://github.com/aws/bedrock-agentcore-sdk-python)를 사용할 것입니다.

In [14]:
import logging
from boto3.session import Session

from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

from lab_helpers.utils import put_ssm_parameter

boto_session = Session()
REGION = boto_session.region_name

logger = logging.getLogger(__name__)

### 단계 2: Bedrock AgentCore Memory 리소스 생성

Amazon Bedrock AgentCore Memory는 AI 에이전트를 위한 지속적인 메모리 기능을 제공하는 완전 관리형 서비스입니다.

#### AgentCore Memory 개념:

1. **단기 메모리 (STM)**: 세션 내의 대화 맥락을 즉시 저장합니다.
2. **장기 메모리 (LTM)**: STM을 비동기적으로 처리하여 의미 있는 패턴, 선호도 및 사실을 추출합니다.
3. **메모리 전략**: 정보 추출 및 구성을 위한 다양한 접근 방식:
   - **USER_PREFERENCE**: 고객의 선호도, 행동 및 패턴을 학습합니다.
   - **SEMANTIC**: 유사성 검색을 위해 벡터 임베딩을 사용하여 사실적 정보를 저장합니다.
4. **네임스페이스**: 고객 및 컨텍스트 유형별로 메모리를 논리적으로 그룹화합니다. 다음 두 가지 네임스페이스를 생성합니다:
- `support/customer/{actorId}/preferences/`: 고객 선호도 및 행동 패턴
- `support/customer/{actorId}/semantic/`: 사실적 정보 및 대화 기록

이 구조는 각 고객의 정보가 격리되고 쉽게 검색될 수 있는 멀티 테넌트 메모리를 가능하게 합니다.

#### 메모리 생성 프로세스:

메모리 리소스를 생성하려면 기본 인프라(벡터 데이터베이스, 처리 파이프라인 등)를 프로비저닝해야 합니다. AWS가 백그라운드에서 관리형 서비스를 설정하므로 일반적으로 2~3분이 소요됩니다.

In [15]:
memory_name = "CustomerSupportMemory"

memory_manager = MemoryManager(region_name=REGION)
memory = memory_manager.get_or_create_memory(
    name=memory_name,
    strategies=[
                {
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "CustomerPreferences",
                        "description": "Captures customer preferences and behavior",
                        "namespaces": ["support/customer/{actorId}/preferences/"],
                    }
                },
                {
                    StrategyType.SEMANTIC.value: {
                        "name": "CustomerSupportSemantic",
                        "description": "Stores facts from conversations",
                        "namespaces": ["support/customer/{actorId}/semantic/"],
                    }
                },
            ]
)
memory_id = memory["id"]
put_ssm_parameter("/app/customersupport/agentcore/memory_id", memory_id)

✅ MemoryManager initialized for region: us-west-2
Memory already exists. Using existing memory ID: CustomerSupportMemory-Wv740S2hyu
🔎 Retrieving memory resource with ID: CustomerSupportMemory-Wv740S2hyu...
  Found memory: CustomerSupportMemory-Wv740S2hyu
Existing {'type': 'SEMANTIC', 'name': 'CustomerSupportSemantic', 'description': 'Stores facts from conversations', 'namespaces': ['support/customer/{actorId}/semantic/']}
Requested {'type': 'SEMANTIC', 'name': 'CustomerSupportSemantic', 'description': 'Stores facts from conversations', 'namespaces': ['support/customer/{actorId}/semantic/']}
Existing {'type': 'USER_PREFERENCE', 'name': 'CustomerPreferences', 'description': 'Captures customer preferences and behavior', 'namespaces': ['support/customer/{actorId}/preferences/']}
Requested {'type': 'USER_PREFERENCE', 'name': 'CustomerPreferences', 'description': 'Captures customer preferences and behavior', 'namespaces': ['support/customer/{actorId}/preferences/']}
Universal strategy valida

In [16]:
if memory_id:
    print("✅ AgentCore Memory created successfully!")
    print(f"Memory ID: {memory_id}")
else:
    print("Memory resource not created. Try Again !")

✅ AgentCore Memory created successfully!
Memory ID: CustomerSupportMemory-Wv740S2hyu


## 단계 3: 이전 고객 상호 작용 시드(Seed) 입력

**왜 메모리를 시딩하나요?**

프로덕션 환경에서 에이전트는 고객 상호 작용을 통해 자연스럽게 메모리를 축적합니다. 하지만 이 랩에서는 실제 대화를 기다리지 않고 장기 메모리(LTM)가 어떻게 작동하는지 보여주기 위해 과거 대화를 시딩합니다.

**메모리 처리 방식:**
1. `create_event`는 상호 작용을 **단기 메모리** (STM)에 즉시 저장합니다.
2. STM은 **장기 메모리** 전략에 의해 비동기적으로 처리됩니다.
3. LTM은 향후 검색을 위해 패턴, 선호도 및 사실을 추출합니다.

이 과정을 확인하기 위해 고객 기록을 시딩해 봅시다:

In [17]:
from lab_helpers.lab2_memory import ACTOR_ID


# Seed with previous customer interactions
previous_interactions = [
    ("I'm having issues with my MacBook Pro overheating during video editing.", "USER"),
    (
        "I can help with that thermal issue. For video editing workloads, let's check your Activity Monitor and adjust performance settings. Your MacBook Pro order #MB-78432 is still under warranty.",
        "ASSISTANT",
    ),
    (
        "What's the return policy on gaming headphones? I need low latency for competitive FPS games",
        "USER",
    ),
    (
        "For gaming headphones, you have 30 days to return. Since you're into competitive FPS, I'd recommend checking the audio latency specs - most gaming models have <40ms latency.",
        "ASSISTANT",
    ),
    (
        "I need a laptop under $1200 for programming. Prefer 16GB RAM minimum and good Linux compatibility. I like ThinkPad models.",
        "USER",
    ),
    (
        "Perfect! For development work, I'd suggest looking at our ThinkPad E series or Dell XPS models. Both have excellent Linux support and 16GB RAM options within your budget.",
        "ASSISTANT",
    ),
]

# Save previous interactions
if memory_id:
    try:
        memory_client = MemoryClient(region_name=REGION)
        memory_client.create_event(
            memory_id=memory_id,
            actor_id=ACTOR_ID,
            session_id="previous_session",
            messages=previous_interactions,
        )
        print("✅ Seeded customer history successfully")
        print("📝 Interactions saved to Short-Term Memory")
        print("⏳ Long-Term Memory processing will begin automatically...")
    except Exception as e:
        print(f"⚠️ Error seeding history: {e}")

✅ Seeded customer history successfully
📝 Interactions saved to Short-Term Memory
⏳ Long-Term Memory processing will begin automatically...


### 메모리 처리 이해하기

`create_event`로 이벤트를 생성한 후, AgentCore Memory는 두 단계로 데이터를 처리합니다:

1. **즉시**: 메시지가 단기 메모리(STM)에 저장됨
2. **비동기**: STM이 장기 메모리(LTM) 전략으로 처리됨

LTM 처리는 시스템이 다음을 수행함에 따라 일반적으로 20~30초가 소요됩니다:
- 대화 패턴 분석
- 고객 선호도 및 행동 추출
- 사실적 정보에 대한 시맨틱 임베딩 생성
- 효율적인 검색을 위해 네임스페이스별로 메모리 구성

고객 선호도를 검색하여 장기 메모리 처리가 완료되었는지 확인해 봅시다:

In [18]:
import time

# Wait for Long-Term Memory processing to complete
print("🔍 Checking for processed Long-Term Memories...")
retries = 0
max_retries = 6  # 1 minute wait

while retries < max_retries:
    memories = memory_client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"support/customer/{ACTOR_ID}/preferences/",
        query="can you summarize the support issue",
    )

    if memories:
        print(
            f"✅ Found {len(memories)} preference memories after {retries * 10} seconds!"
        )
        break

    retries += 1
    if retries < max_retries:
        print(
            f"⏳ Still processing... waiting 10 more seconds (attempt {retries}/{max_retries})"
        )
        time.sleep(10)
    else:
        print(
            "⚠️ Memory processing is taking longer than expected. This can happen with overloading.."
        )
        break

print(
    "🎯 AgentCore Memory automatically extracted these customer preferences from our seeded conversations:"
)
print("=" * 80)

for i, memory in enumerate(memories, 1):
    if isinstance(memory, dict):
        content = memory.get("content", {})
        if isinstance(content, dict):
            text = content.get("text", "")
            print(f"  {i}. {text}")

🔍 Checking for processed Long-Term Memories...
✅ Found 3 preference memories after 0 seconds!
🎯 AgentCore Memory automatically extracted these customer preferences from our seeded conversations:
  1. {"context":"The user explicitly stated preference for good Linux compatibility in laptops.","preference":"Prefers laptops with good Linux compatibility","categories":["technology","computers","operating systems","Linux"]}
  2. {"context":"The user explicitly mentioned needing low latency gaming headphones for competitive FPS games, directly stating their gaming interest and requirements.","preference":"Plays competitive FPS games and requires low latency audio equipment","categories":["gaming","entertainment","audio equipment","FPS games"]}
  3. {"context":"The user explicitly stated they are experiencing MacBook Pro overheating issues during video editing, indicating they perform video editing work.","preference":"Uses MacBook Pro for video editing","categories":["technology","computers",

### 시맨틱 메모리 탐색

시맨틱 메모리는 대화에서 얻은 사실적 정보를 벡터 임베딩을 사용하여 저장합니다. 이를 통해 관련 사실과 맥락을 유사성 기반으로 검색할 수 있습니다.

In [6]:
import time

# Retrieve semantic memories (factual information)
while True:
    semantic_memories = memory_client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"support/customer/{ACTOR_ID}/semantic/",
        query="information on the technical support issue",
    )
    print("🧠 AgentCore Memory identified these factual details from conversations:")
    print("=" * 80)
    if semantic_memories:
        break
    time.sleep(10)
for i, memory in enumerate(semantic_memories, 1):
    if isinstance(memory, dict):
        content = memory.get("content", {})
        if isinstance(content, dict):
            text = content.get("text", "")
            print(f"  {i}. {text}")

🧠 AgentCore Memory identified these factual details from conversations:
  1. The user's MacBook Pro order number is #MB-78432 and is still under warranty.
  2. The user plays competitive FPS games and needs low latency audio equipment.
  3. The user is looking for a laptop under $1200 for programming work.


## 단계 4: 메모리가 있는 고객 지원 에이전트 생성

다음으로, Lab 1에서 했던 것처럼 고객 지원 에이전트를 구현하겠지만, 이번에는 에이전트의 세션 관리자에 AgentCoreMemoryConfig를 추가합니다. 이를 통해 에이전트가 새로운 이벤트를 생성하고 메모리를 검색할 수 있게 됩니다.

In [19]:
import uuid

from strands import Agent
from strands.models import BedrockModel
from bedrock_agentcore.memory.integrations.strands.config import AgentCoreMemoryConfig, RetrievalConfig
from bedrock_agentcore.memory.integrations.strands.session_manager import AgentCoreMemorySessionManager

from lab_helpers.lab1_strands_agent import (
    SYSTEM_PROMPT,
    get_return_policy,
    web_search,
    get_product_info,
    get_technical_support,
    MODEL_ID,
)

session_id = uuid.uuid4()

memory_config = AgentCoreMemoryConfig(
        memory_id=memory_id,
        session_id=str(session_id),
        actor_id=ACTOR_ID,
        retrieval_config={
            "support/customer/{actorId}/semantic/": RetrievalConfig(top_k=3, relevance_score=0.2),
            "support/customer/{actorId}/preferences/": RetrievalConfig(top_k=3, relevance_score=0.2)
        }
    )

# Initialize the Bedrock model (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(model_id=MODEL_ID, region_name=REGION)

# Create the customer support agent with all 5 tools
agent = Agent(
    model=model,
    session_manager=AgentCoreMemorySessionManager(memory_config, REGION),
    tools=[
        get_product_info,  # Tool 1: Simple product information lookup
        get_return_policy,  # Tool 2: Simple return policy lookup
        web_search,
        get_technical_support,
    ],
    system_prompt=SYSTEM_PROMPT,
)

## 단계 5: 개인화된 에이전트 테스트

메모리가 강화된 에이전트를 테스트해 봅시다! 에이전트가 고객의 과거 선호도를 사용하여 개인화된 추천을 제공하는 모습을 지켜보세요.

에이전트는 자동으로 다음을 수행합니다:
1. 메모리에서 관련 고객 컨텍스트 검색
2. 해당 컨텍스트를 사용하여 응답 개인화
3. 나중에 사용할 수 있도록 이 새로운 상호 작용 저장

In [20]:
print("🎧 Testing headphone recommendation with customer memory...\n\n")
response1 = agent("Which headphones would you recommend?")

🎧 Testing headphone recommendation with customer memory...




Great question! To give you the best headphone recommendation, I'd like to understand your needs better. Based on your profile, I can see you're interested in competitive FPS gaming with low latency audio equipment.

Here are some key questions that would help me recommend the perfect headphones for you:

1. **Primary Use**: Are you looking for headphones mainly for competitive gaming, or do you also need them for general use (music, movies, work)?

2. **Connection Type**: Do you prefer:
   - Wired (typically lower latency for gaming)
   - Wireless 2.4GHz dongle (gaming-optimized)
   - Bluetooth
   - Or are you open to any?

3. **Budget**: What's your price range?

4. **Key Features**: Which matters most to you?
   - Low latency/response time
   - Comfortable for long sessions
   - Surround sound capabilities
   - Noise cancellation
   - Durability

5. **Platforms**: Will you mainly use these with your laptop, or do you als

In [13]:
print("\n💻 Testing laptop preference recall...\n\n")
response2 = agent("What is my preferred laptop brand and requirements?")


💻 Testing laptop preference recall...




Based on your preferences, here's what I have on file for you:

**Your Preferred Laptop Brand:** **ThinkPad** (Lenovo)

**Your Requirements:**
- **RAM:** Minimum 16GB
- **Operating System Compatibility:** Good Linux compatibility
- **Use Case:** Programming work
- **Budget:** Under $1,200

This profile makes perfect sense! ThinkPad models are known for:
- Excellent build quality and durability
- Strong Linux compatibility (many ThinkPad users prefer Linux)
- Solid performance for programming and development work
- Good keyboard for extensive typing
- Professional design

Is there anything specific you'd like to know about:
- Current ThinkPad models that meet your specifications?
- Linux compatibility details for specific ThinkPad models?
- Warranty or return policy information?
- Technical specifications of a particular model?

I'm here to help you find the perfect laptop for your programming needs!

## 축하합니다! 🎉

**Lab 2: 고객 지원 에이전트에 메모리 추가**를 성공적으로 완료했습니다!

### 달성한 내용:

- Amazon Bedrock AgentCore Memory로 서버리스 관리형 메모리 생성
- 사용자 선호도 및 시맨틱(사실적) 정보를 저장하기 위한 장기 메모리 구현
- Strands Agents에서 제공하는 세션 관리 메커니즘을 사용하여 AgentCore Memory를 고객 지원 에이전트와 통합

##### 다음 [Lab 3 - 게이트웨이 및 아이덴티티로 확장  →](lab-03-agentcore-gateway.ipynb)

## 리소스
- [Amazon Bedrock Agent Core Memory](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory.html)
- [Amazon Bedrock AgentCore Memory 심층 분석 블로그](https://aws.amazon.com/blogs/machine-learning/amazon-bedrock-agentcore-memory-building-context-aware-agents/)
- [Strands Agent SDK - AgentCore Memory 예제](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/strands-sdk-memory.html)